In [7]:
#Importing libraries

import json
import csv
import matplotlib
import numpy as np
from os import listdir
from sklearn import linear_model

In [8]:
# The order of La liga at the end of the season
#The idea is to use this to predict the result of the matches, ie Barcelona won all of them and levante won none
order ={}
order[1] = 'barcelona'
order[2] = 'real madrid'
order[3] = 'atl madrid'
order[4] = 'villarreal'
order[5] = 'ath bilbao'
order[6] = 'celta vigo'
order[7] = 'sevilla'
order[8] = 'malaga-cf'
order[9] = 'real sociedad'
order[10] = 'real betis'
order[11] = 'las palmas'
order[12] = 'valencia'
order[13] = 'espanyol'
order[14] = 'sd-eibar'
order[15] = 'deportivo'
order[16] = 'granada'
order[17] = 'sporting gijon'
order[18] = 'rayo vallecano'
order[19] = 'getafe'
order[20] = 'levante'

#getting the position of the team
def get_position(team):
    min_lev = [30, 99] #position, distance lev
    for key,t in order.iteritems():
        dist = lev(team,t)
        if dist<= min_lev[1]:
            min_lev[0] = key
            min_lev[1] = dist
    return min_lev[0]
    

#get_position('espanol')


In [9]:
#this is necessary because of the accents an other errors from scrapping around
def levenshtein(s1, s2):
    #https://en.wikipedia.org/wiki/Levenshtein_distance
    if len(s1) < len(s2):
        return lev(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def lev(s1,s2):
    #convert to lower everything to avoid mistakes
    return levenshtein(s1.lower(),s2.lower())

In [10]:
#Reading the bets and other info
bets = np.genfromtxt('/home/roc/workspace/fifa-db-players/match-results/1516.csv', delimiter=',', skip_header=1
                                ,dtype="|S10")

#matches
matches = bets[:,2:4]
aux = bets[:,6]
results = []
for r in aux:
    if r == 'D':
        results.append(0)
    elif r == 'A':
        results.append(1)
    else: #H
        results.append(-1)


In [11]:
#Result match my encoding
#homies wins -1
#tie 0
#outer wins 1

my_results_basic = []
for match in matches:
    p1 = get_position(match[0])
    p2 = get_position(match[1])

    #with only win/lose it gets 1% improvement
    if p1+1 == p2 or p2+1==p1:
        my_results_basic.append(0)
    elif p1<p2:
        my_results_basic.append(-1)
    else:
        my_results_basic.append(1)


def get_accuracy(results, my_results_basic):
    count = 0.0
    for i in range(len(results)):
        if results[i] == my_results_basic[i]:
            count +=1
    return count
print "Based on the final Liga order accuracy", get_accuracy(results, my_results_basic)/len(results)*100.0,'%'

Based on the final Liga order accuracy 48.4210526316 %


In [12]:
#Get the prediction from the different betting webs

#returns the list witht the bets results
def get_results_from_bets(bets):
    results = []
    for bet in bets:
        if(bet[0] < bet[1] and bet[0] < bet[2]) :
            results.append(-1)
        elif(bet[1] < bet[0] and bet[1] < bet[2]) :
            results.append(0)
        elif(bet[2] < bet[0] and bet[2] < bet[1]) :
            results.append(1)
        else:
            results.append(666666)
    return results

for i in range(13):
    print 'bet odds', get_accuracy(results, get_results_from_bets(bets[:,22+i*3:25+i*3]))/len(results)*100.0,'%'


bet odds 54.2105263158 %
bet odds 55.0 %
bet odds 52.8947368421 %
bet odds 54.2105263158 %
bet odds 54.4736842105 %
bet odds 53.6842105263 %
bet odds 54.4736842105 %
bet odds 25.5263157895 %
bet odds 30.2631578947 %
bet odds 21.8421052632 %
bet odds 40.2631578947 %
bet odds 25.2631578947 %
bet odds 38.6842105263 %


In [18]:
#TODO
#Getting the club players info
clubs = []
c_names = listdir('/home/roc/workspace/fifa-db-players/club-data')
for key, value in order.iteritems():
    min_lev = [30, 99] #position, distance lev
    for c in c_names:
        dist = lev(value,c[:-5])
        if dist<= min_lev[1]:
            min_lev[0] = c
            min_lev[1] = dist
    clubs.append(min_lev[0])

with open('/home/roc/workspace/fifa-db-players/club-data/fc-barcelona.json') as data_file:    
    data = json.load(data_file)

i = 0
for player_id in data:
    print data[player_id]['name']
    


rafinha
marc-andre-ter-stegen
lionel-messi
pique
luis-suarez
javier-mascherano
iniesta
adriano
thomas-vermaelen
neymar
claudio-bravo
munir
jeremy-mathieu
xavi
jordi-masip
sergi-roberto
douglas
dani-alves
ivan-rakitic
sandro
pedro
marc-bartra
montoya
jordi-alba
sergio-busquets


In [2]:
s = 'abcdefghijkl'
print s[:-3]

abcdefghi
